

### Che cos'è lo Stacking (Stacked Generalization)?

Immagina di avere un problema difficile e di chiedere consiglio a tre esperti diversi:

1.  Un matematico (XGBoost)
2.  Un fisico (CatBoost)
3.  Un ingegnere (LightGBM)

Ognuno ti dà una risposta leggermente diversa.
Invece di fare una media semplice (che sarebbe il *Voting* o *Averaging*), assumi un **Manager** (il Meta-Learner). Il Manager ha studiato come si sono comportati gli esperti in passato e sa che: *"Quando il matematico dice X e il fisico dice Y, di solito ha ragione il fisico, ma l'ingegnere corregge l'errore sui casi limite".*

Il Manager impara a **pesare** le previsioni degli esperti per dare la risposta finale ottimale.

### Come funziona 

Nello script, lo stacking avviene in due livelli (Level 0 e Level 1).

#### Livello 0: I "Base Learners" (Gli Esperti)

Questi sono i modelli forti che hai ottimizzato con Optuna. Nel tuo codice sono definiti qui:

```python
estimators = [
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params)),
    ('lgbm', LGBMRegressor(**best_lgbm_params))
]
```

Questi modelli guardano i dati originali (`X_train_full`) e cercano di predire il prezzo delle case (`y`). Essendo tre varianti di Gradient Boosting, sono tutti molto potenti, ma faranno errori leggermente diversi su case diverse.

#### Livello 1: Il "Meta Learner" (Il Manager)

Questo è il modello che prende le decisioni finali. Nel tuo codice è:

```python
final_estimator = RidgeCV()
```

Viene scelta una **Ridge Regression** (regressione lineare con regolarizzazione L2). Questa è una scelta **standard** per lo stacking.

**Perché RidgeCV?**
I modelli di base (XGB, Cat, LGBM) sono molto complessi e non lineari. Il Meta-Learner non deve imparare di nuovo i pattern complessi dei dati; deve solo imparare **come combinare linearmente** le predizioni dei tre modelli sopra. Una regressione semplice evita che il Meta-Learner vada in overfitting sulle predizioni.

### Il processo "Sotto il cofano" (Cross-Validation)

Nel codice:

```python
stacking = StackingRegressor(..., cv=5, ...)
```

Questo `cv=5` è cruciale. Ecco cosa succede quando lanci `stacking.fit`:

1.  **Generazione delle Meta-Feature (Out-of-Fold Predictions):**

      * Il dataset di train viene diviso in 5 parti (folds).
      * I modelli base (XGB, Cat, LGBM) vengono addestrati su 4 parti e fanno previsioni sulla 5ª parte (che non hanno mai visto).
      * Si ripete per tutte le 5 parti.
      * Alla fine, per ogni riga del tuo dataset di train, hai 3 previsioni (una per modello) che sono state generate senza "vedere" quel dato durante il training. Queste 3 previsioni diventano le nuove colonne (Meta-Features) per il Meta-Learner.

2.  **Addestramento Meta-Learner:**

      * Il `RidgeCV` viene addestrato su queste 3 nuove colonne per predire il target reale. Impara i coefficienti (i pesi) da dare a ciascun modello.

3.  **Retraining Finale:**

      * Una volta che il `RidgeCV` è addestrato, i modelli base (XGB, Cat, LGBM) vengono ri-addestrati su **tutto** il dataset `X_train_full` per essere pronti a fare previsioni sui nuovi dati (`X_test`).

### Perché questo approccio è vincente

1.  **Diversity:** Anche usando tre librerie di Boosting (che sono simili), hanno implementazioni diverse (CatBoost gestisce meglio le categoriche/numeriche, XGBoost è aggressivo, LightGBM è veloce e usa leaf-wise growth). I loro errori sono spesso decorrelati.
2.  **Robustezza:** La RidgeCV come meta-learner "liscia" le predizioni. Se XGBoost impazzisce su una casa e predice un valore assurdo, ma CatBoost e LightGBM sono stabili, la Ridge ridurrà l'impatto dell'errore di XGBoost.
3.  **Ottimizzazione Precedente:** Si passano i `best_params` da Optuna. Stai facendo stacking di modelli *già ottimizzati*, non di modelli a caso. Questo massimizza il guadagno.

### Sintesi dei Risultati

Lo stacking quasi sempre batte il miglior modello singolo, solitamente riducendo l'RMSE di un ulteriore 0.5% - 1.5%. Nel tuo output finale vedrai che l'RMSE dello stacking sarà probabilmente inferiore al "BEST VAL RMSE" che hai stampato per i singoli modelli Optuna.



In [ ]:
import numpy as np
import pandas as pd
import optuna
import warnings
import itertools
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset e Preprocessing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectFromModel

# Modelli
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from catboost import CatBoostRegressor
from sklearn.linear_model import RidgeCV
from sklearn.ensemble import StackingRegressor

# Configurazione
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. CARICAMENTO E PULIZIA DATI
# ---------------------------------------------------------
print("1. Caricamento e Pulizia Dati...")
start_global = time.time()

data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="MedHouseVal")
df = pd.concat([X, y], axis=1)

# Rimuovi il cap a 5.0 (valore massimo del dataset originale che crea distorsioni)
df = df[df['MedHouseVal'] < 5.0]

def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    indices_to_drop = []
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df_clean[(df_clean[col] < Q1 - 2.0*IQR) | (df_clean[col] > Q3 + 2.0*IQR)].index
        indices_to_drop.extend(outliers)
    return df_clean.drop(list(set(indices_to_drop)))

cols_clean = ['AveRooms', 'AveBedrms', 'AveOccup', 'MedInc']
df = remove_outliers_iqr(df, cols_clean)
print(f"   -> Dataset pulito: {df.shape}")

# ---------------------------------------------------------
# 2. FEATURE ENGINEERING MASSIVA
# ---------------------------------------------------------
print("2. Generazione Feature 'Combo' (Multiplicazioni, Divisioni, Log)...")

def generate_comprehensive_features(df_input, cols_to_combine):
    df_eng = df_input.copy()
    math_cols = [c for c in cols_to_combine if c not in ['Latitude', 'Longitude', 'Geo_Cluster']]
    
    print(f"   -> Generazione combinazioni su {len(math_cols)} colonne base...")

    # A. LOGARITMI
    for col in math_cols:
        if df_eng[col].min() >= 0:
            df_eng[f'LOG_{col}'] = np.log1p(df_eng[col])

    # B. MOLTIPLICAZIONI
    for col1, col2 in itertools.combinations(math_cols, 2):
        col_name = f'MULT_{col1}_x_{col2}'
        df_eng[col_name] = df_eng[col1] * df_eng[col2]

    # C. DIVISIONI / RAPPORTI
    for col1, col2 in itertools.permutations(math_cols, 2):
        col_name = f'RATIO_{col1}_div_{col2}'
        df_eng[col_name] = df_eng[col1] / (df_eng[col2] + 1e-5)

    return df_eng

# --- Geo Features Base ---
sf_coords = (37.7749, -122.4194)
la_coords = (34.0522, -118.2437)
df['Dist_SF'] = np.sqrt((df['Latitude'] - sf_coords[0])**2 + (df['Longitude'] - sf_coords[1])**2)
df['Dist_LA'] = np.sqrt((df['Latitude'] - la_coords[0])**2 + (df['Longitude'] - la_coords[1])**2)

coords = df[['Latitude', 'Longitude']]
# KMeans (Cluster geografici)
kmeans = KMeans(n_clusters=15, random_state=42, n_init=10)
df['Geo_Cluster'] = kmeans.fit_predict(StandardScaler().fit_transform(coords))

X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

cols_for_math = [c for c in X.columns if c != 'Geo_Cluster']
X_full = generate_comprehensive_features(X, cols_for_math)

# Pulizia inf/nan generati dalle divisioni
X_full.replace([np.inf, -np.inf], np.nan, inplace=True)
X_full.fillna(0, inplace=True)

print(f"   -> Totale Feature Generate: {X_full.shape[1]}")

# ---------------------------------------------------------
# 3. IL GIUDICE: SELEZIONE FEATURE CON XGBOOST GPU
# ---------------------------------------------------------
print("3. Selezione Feature tramite GPU...")

scaler = RobustScaler()
# Nota: La selezione feature non necessita di early stopping complesso
selector_model = XGBRegressor(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    tree_method='gpu_hist', 
    device='gpu',
    n_jobs=1,
    random_state=42,
    verbose=0
)
selector_model.fit(X_full, y)

selection = SelectFromModel(selector_model, prefit=True, threshold='1.25*median')

X_selected = X_full.loc[:, selection.get_support()]
print(f"   -> Feature Sopravvissute: {X_selected.shape[1]}")

selected_names = X_full.columns[selection.get_support()]
print(f"   -> Esempio Top Feature scelte: {list(selected_names[:5])}...")

X_train_full, X_test, y_train_full, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# =========================================================
# === SPLIT PER OPTUNA VELOCE (HOLD-OUT) ===
# =========================================================
X_opt_train, X_opt_val, y_opt_train, y_opt_val = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=42
)
print(f"\nSet Ottimizzazione: Train={X_opt_train.shape[0]}, Validation={X_opt_val.shape[0]}")

# ---------------------------------------------------------
# 4. OPTUNA SUPER VELOCE (Aggiornato per nuove API)
# ---------------------------------------------------------
print("\n4. Tuning Iperparametri VELOCE (Hold-out validation con GPU)...")
N_TRIALS = 50 

# --- A. XGBoost (GPU - Aggiornato) ---
def objective_xgb(trial):
    params = {
        'n_estimators': 4000, 
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
        'max_depth': trial.suggest_int('max_depth', 4, 12),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10),
        'tree_method': 'gpu_hist',
        'device': 'gpu',
        'n_jobs': 1,
        'random_state': 42,
        'eval_metric': 'rmse',
        # FIX: early_stopping_rounds va qui nel costruttore per le nuove versioni
        'early_stopping_rounds': 100 
    }
    
    # Inizializza con early_stopping_rounds
    model = XGBRegressor(**params)
    

    model.fit(
        X_opt_train, y_opt_train,
        eval_set=[(X_opt_val, y_opt_val)],
        verbose=False # In XGBoost 2.0+ a volte verbose in fit è deprecato, ma False di solito passa
    )
    
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print(" -> Optimizing XGBoost...")
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=N_TRIALS)

# --- B. CatBoost (GPU) ---
def objective_cat(trial):
    params = {
        'iterations': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
        'depth': trial.suggest_int('depth', 6, 13), 
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'task_type': 'GPU',
        'devices': '0',
        'verbose': 0,
        'allow_writing_files': False,
        'random_state': 42,
        'eval_metric': 'RMSE'
    }
    model = CatBoostRegressor(**params)
    
    # CatBoost accetta ancora early_stopping_rounds in fit
    model.fit(
        X_opt_train, y_opt_train,
        eval_set=[(X_opt_val, y_opt_val)],
        early_stopping_rounds=100
    )
    
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print(" -> Optimizing CatBoost...")
study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=N_TRIALS) 

# --- C. LightGBM (Hybrid - Aggiornato) ---
def objective_lgbm(trial):
    params = {
        'n_estimators': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1),
        'num_leaves': trial.suggest_int('num_leaves', 30, 200),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.95),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.95),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.01, 10),
        'device': 'gpu',
        'n_jobs': 1,
        'verbosity': -1, # Usa verbosity qui invece di verbose
        'random_state': 42,
        'metric': 'rmse'
    }
    
    try:
        model = LGBMRegressor(**params)
        # FIX: Usa callbacks espliciti, rimuovi early_stopping_rounds dagli argomenti diretti
        model.fit(
            X_opt_train, y_opt_train,
            eval_set=[(X_opt_val, y_opt_val)],
            eval_metric='rmse',
            callbacks=[early_stopping(stopping_rounds=100, verbose=False), log_evaluation(period=0)]
        )
        preds = model.predict(X_opt_val)
        return np.sqrt(mean_squared_error(y_opt_val, preds))
    
    except Exception as e:
        # Fallback CPU se GPU non va o errore specifico
        # print(f"LGBM GPU Error: {e}") 
        params['device'] = 'cpu'
        params['n_jobs'] = -1 
        model = LGBMRegressor(**params)
        model.fit(
            X_opt_train, y_opt_train,
            eval_set=[(X_opt_val, y_opt_val)],
            eval_metric='rmse',
            callbacks=[early_stopping(stopping_rounds=100, verbose=False), log_evaluation(period=0)]
        )
        preds = model.predict(X_opt_val)
        return np.sqrt(mean_squared_error(y_opt_val, preds))

print(" -> Optimizing LightGBM...")
study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=N_TRIALS)

print(f"\nBEST VAL RMSE:")
print(f"XGB: {study_xgb.best_value:.4f}")
print(f"CAT: {study_cat.best_value:.4f}")
print(f"LGBM: {study_lgbm.best_value:.4f}")

# ---------------------------------------------------------
# 5. STACKING FINALE (Ensemble)
# ---------------------------------------------------------
print("\n5. Addestramento Stacking Ensemble Finale (su set completo)...")

# Recuperiamo i migliori parametri 
best_xgb_params = study_xgb.best_params
# Assicuriamoci che non ci siano parametri di fit residui
best_xgb_params.update({
    'tree_method': 'gpu_hist', 'device': 'gpu', 'n_jobs': 1, 
    'random_state': 42, 'early_stopping_rounds': None # Disabilita early stopping per il fit finale
})

best_cat_params = study_cat.best_params
best_cat_params.update({
    'task_type': 'GPU', 'devices': '0', 'verbose': 0, 
    'allow_writing_files': False, 'random_state': 42
})

best_lgbm_params = study_lgbm.best_params
if study_lgbm.best_params.get('device') == 'cpu':
    best_lgbm_params.update({'n_jobs': -1, 'verbosity': -1, 'random_state': 42})
else:
    best_lgbm_params.update({'device': 'gpu', 'n_jobs': 1, 'verbosity': -1, 'random_state': 42})

estimators = [
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params)),
    ('lgbm', LGBMRegressor(**best_lgbm_params))
]

# Stacking Regressor non supporta nativamente early stopping nei sotto-modelli
# durante il cross-validation interno in modo semplice, quindi usiamo n_estimators ottimizzati
# (Nota: idealmente dovresti settare n_estimators al valore ottimo trovato durante Optuna, 
# ma qui li lasciamo alti o default per lo stacking standard)

stacking = StackingRegressor(
    estimators=estimators,
    final_estimator=RidgeCV(),
    cv=5,
    n_jobs=1 
)

stacking.fit(X_train_full, y_train_full)

# ---------------------------------------------------------
# 6. VALUTAZIONE FINALE
# ---------------------------------------------------------
y_pred = stacking.predict(X_test)
final_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
final_r2 = r2_score(y_test, y_pred)
total_time = time.time() - start_global

print(f"\n==========================================")
print(f" RISULTATI FINALI (CODICE AGGIORNATO)")
print(f"==========================================")
print(f" RMSE: {final_rmse:.5f}")
print(f" R^2 : {final_r2:.5f}")
print(f" Tempo Totale: {total_time:.2f} sec")
print(f"==========================================")

# Plot veloce importanza feature (su XGBoost)
final_xgb = XGBRegressor(**best_xgb_params)
final_xgb.fit(X_train_full, y_train_full)
plt.figure(figsize=(10, 8))
feat_importances = pd.Series(final_xgb.feature_importances_, index=X_train_full.columns)
feat_importances.nlargest(20).plot(kind='barh')
plt.title("Top 20 Feature Importanti (XGBoost GPU)")
plt.tight_layout()
plt.show()



### 1. Il Primo Stadio: `SmartSoftModelSelector` (Il Cervello)

Nel codice precedente si usava una `RidgeCV` che assegnava pesi fissi (es. 0.4 a XGB, 0.4 a Cat, 0.2 a LGBM).
Qui invece si crea una **Gating Network** dinamica.

* **Il Concetto:** Non tutti i modelli sono bravi ovunque. Forse XGBoost è bravissimo con le case costose, mentre CatBoost è più preciso con le case vecchie.
* **Come funziona nel codice:**
    1.  Addestri i 3 modelli base ("Gli Esperti").
    2.  Calcoli chi ha sbagliato di meno per ogni singola casa del training set (`y_best_model_idx`).
    3.  Addestri un classificatore (`self.selector_model`, un XGBClassifier) che prende le caratteristiche della casa e predice: *"Quale esperto devo ascoltare per questa specifica casa?"*.
    4.  **Soft Voting:** Non scegli solo un vincitore. Se il selettore dice: "80% probabilità che XGBoost abbia ragione, 20% CatBoost", la predizione finale sarà una media pesata 80/20.



---

### 2. Il Secondo Stadio: `ResidualCorrectedMoE` (Il Modello dei Bias)

È una tecnica potente spesso usata nelle competizioni Kaggle o nei sistemi finanziari per correggere errori sistematici.

#### La Logica Matematica
Normalmente, l'equazione è:
$$\text{Predizione} = f(X)$$
Nel tuo codice, l'equazione diventa:
$$\text{Predizione Finale} = f_{MoE}(X) + g_{Correttore}(X)$$

Dove $g(X)$ stima l'errore commesso da $f(X)$.

#### Cosa fa il codice riga per riga:

1.  **`base_moe_model.fit(X, y)`**:
    Il primo stadio fa il grosso del lavoro e impara il prezzo delle case.

2.  **`cross_val_predict(..., cv=5)` (Cruciale!)**:
    Qui sta la magia. Il codice genera le predizioni sul training set usando la cross-validation.
    * *Perché non usare `.predict()` normale?* Se usassi predict sui dati su cui hai appena addestrato, l'errore sarebbe bassissimo (overfitting). Usando `cross_val_predict`, simuli come il modello si comporta su dati **mai visti**.

3.  **`residuals = y - oof_preds`**:
    Calcoli l'errore.
    * Se `residual` è positivo (+10k), significa che il modello base ha **sottostimato** il prezzo.
    * Se `residual` è negativo (-10k), ha **sovrastimato**.

4.  **`corrector_model.fit(X, residuals)`**:
    Addestri un nuovo modello (CatBoost leggero) non a predire il prezzo, ma a predire **di quanto sbaglierà il primo modello**.
    Il correttore impara i **Bias Sistematici**.
    * *Esempio:* "Ogni volta che la casa è a San Francisco (`Dist_SF` basso) e ha 2 bagni, il modello principale tende a sottostimare di 15.000$". Il correttore impara questo pattern e aggiungerà automaticamente 15.000$.



### Perché questo approccio è rischioso ma geniale?

1.  **Geniale:** Rimuovi le distorsioni strutturali. Se i tuoi modelli base sono tutti "prudenti" sui prezzi alti, il correttore imparerà ad essere "aggressivo" per compensare.
2.  **Rischioso (Overfitting):** C'è il rischio che il correttore impari il "rumore" invece del bias reale.
    * *La tua protezione:* Nel codice vedo che hai configurato il `corrector_model` con parametri conservativi (`depth=6`, `l2_leaf_reg=5`, `iterations=500`). Questo è corretto: il correttore deve essere "debole" per catturare solo i trend generali dell'errore e non memorizzare il dataset.

### Sintesi Grafica della tua Pipeline

Possiamo riassumere il flusso dei dati nel tuo script così:

1.  **Input Dati**
2.  **Livello 1 (Esperti):** XGB + Cat + LGBM generano 3 opinioni.
3.  **Livello 1.5 (Gating):** Il Classificatore decide quanto fidarsi di ognuno -> Esce `Base Prediction`.
4.  **Livello 2 (Correttore):** Il CatBoost dei Residui guarda i dati e dice: "Attenzione, qui stiamo sottostimando". -> Esce `Correction`.
5.  **Output Finale:** `Base Prediction` + `Correction`.



In [ ]:
import numpy as np
import pandas as pd
import optuna
import warnings
import itertools
import time
import matplotlib.pyplot as plt
import seaborn as sns

# Dataset e Preprocessing
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.cluster import KMeans
from sklearn.feature_selection import SelectFromModel
from sklearn.base import BaseEstimator, RegressorMixin

# Modelli
from xgboost import XGBRegressor, XGBClassifier
from lightgbm import LGBMRegressor, early_stopping, log_evaluation
from catboost import CatBoostRegressor

# Configurazione
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings('ignore')

# ---------------------------------------------------------
# 1. CARICAMENTO E PULIZIA DATI
# ---------------------------------------------------------
print("1. Caricamento e Pulizia Dati...")
start_global = time.time()

data = fetch_california_housing()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name="MedHouseVal")
df = pd.concat([X, y], axis=1)

# Rimuoviamo il cap a 5.0
df = df[df['MedHouseVal'] < 5.0]

def remove_outliers_iqr(df, columns):
    df_clean = df.copy()
    indices_to_drop = []
    for col in columns:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = df_clean[(df_clean[col] < Q1 - 2.0*IQR) | (df_clean[col] > Q3 + 2.0*IQR)].index
        indices_to_drop.extend(outliers)
    return df_clean.drop(list(set(indices_to_drop)))

cols_clean = ['AveRooms', 'AveBedrms', 'AveOccup', 'MedInc']
df = remove_outliers_iqr(df, cols_clean)
print(f"   -> Dataset pulito: {df.shape}")

# ---------------------------------------------------------
# 2. FEATURE ENGINEERING MASSIVA
# ---------------------------------------------------------
print("2. Generazione Feature 'Combo'...")

def generate_comprehensive_features(df_input, cols_to_combine):
    df_eng = df_input.copy()
    math_cols = [c for c in cols_to_combine if c not in ['Latitude', 'Longitude', 'Geo_Cluster']]
    
    # Trucco Rotazione Coordinate per Alberi
    df_eng['Rot_45_LatLon'] = df_eng['Latitude'] + df_eng['Longitude']
    df_eng['Rot_N45_LatLon'] = df_eng['Latitude'] - df_eng['Longitude']

    # A. LOGARITMI
    for col in math_cols:
        if df_eng[col].min() >= 0:
            df_eng[f'LOG_{col}'] = np.log1p(df_eng[col])

    # B. MOLTIPLICAZIONI
    for col1, col2 in itertools.combinations(math_cols, 2):
        col_name = f'MULT_{col1}_x_{col2}'
        df_eng[col_name] = df_eng[col1] * df_eng[col2]

    # C. DIVISIONI
    for col1, col2 in itertools.permutations(math_cols, 2):
        col_name = f'RATIO_{col1}_div_{col2}'
        df_eng[col_name] = df_eng[col1] / (df_eng[col2] + 1e-5)

    return df_eng

# Geo Features Base
sf_coords = (37.7749, -122.4194)
la_coords = (34.0522, -118.2437)
df['Dist_SF'] = np.sqrt((df['Latitude'] - sf_coords[0])**2 + (df['Longitude'] - sf_coords[1])**2)
df['Dist_LA'] = np.sqrt((df['Latitude'] - la_coords[0])**2 + (df['Longitude'] - la_coords[1])**2)

coords = df[['Latitude', 'Longitude']]
kmeans = KMeans(n_clusters=15, random_state=42, n_init=10)
df['Geo_Cluster'] = kmeans.fit_predict(StandardScaler().fit_transform(coords))

X = df.drop('MedHouseVal', axis=1)
y = df['MedHouseVal']

cols_for_math = [c for c in X.columns if c != 'Geo_Cluster']
X_full = generate_comprehensive_features(X, cols_for_math)
X_full.replace([np.inf, -np.inf], np.nan, inplace=True)
X_full.fillna(0, inplace=True)
print(f"   -> Totale Feature Generate: {X_full.shape[1]}")

# ---------------------------------------------------------
# 3. SELEZIONE FEATURE CON XGBOOST GPU
# ---------------------------------------------------------
print("3. Selezione Feature tramite GPU...")
scaler = RobustScaler()
# Nota: fit non richiede scaling per alberi, ma utile averlo per prassi
X_scaled = scaler.fit_transform(X_full)

selector_model = XGBRegressor(
    n_estimators=500, max_depth=8, learning_rate=0.05,
    tree_method='hist', device='gpu', n_jobs=1, random_state=42
)
selector_model.fit(X_full, y)
selection = SelectFromModel(selector_model, prefit=True, threshold='1.25*median')

X_selected = X_full.loc[:, selection.get_support()]
print(f"   -> Feature Sopravvissute: {X_selected.shape[1]}")

# Split Principale (Train / Test)
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

# ---------------------------------------------------------
# 4. OPTUNA REALE CON EARLY STOPPING CORRETTO
# ---------------------------------------------------------
print("\n4. Tuning Iperparametri con Optuna e Early Stopping...")

# Creiamo un set di validazione SOLO per Optuna per monitorare l'early stopping
X_opt_train, X_opt_val, y_opt_train, y_opt_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

N_TRIALS = 20  # Numero di trial (aumentare per risultati migliori)

# --- A. XGBoost Optimization ---
def objective_xgb(trial):
    params = {
        'n_estimators': 4000,  # Alto, tanto ferma l'early stopping
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 0.95),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10.0, log=True),
        'tree_method': 'hist',
        'device': 'gpu',
        'n_jobs': 1,
        'random_state': 42,
        # Early Stopping nel costruttore per versioni recenti (o gestito in fit)
        'early_stopping_rounds': 100
    }
    
    model = XGBRegressor(**params)
    
    # Verbose=False sopprime i log, eval_set serve per l'early stopping
    model.fit(
        X_opt_train, y_opt_train,
        eval_set=[(X_opt_val, y_opt_val)],
        verbose=False
    )
    
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print("   -> Optimizing XGBoost...")
study_xgb = optuna.create_study(direction='minimize')
study_xgb.optimize(objective_xgb, n_trials=N_TRIALS)

# --- B. CatBoost Optimization ---
def objective_cat(trial):
    params = {
        'iterations': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'depth': trial.suggest_int('depth', 6, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 0.95),
        'task_type': 'GPU',
        'devices': '0',
        'verbose': 0,
        'allow_writing_files': False,
        'random_state': 42,
        'eval_metric': 'RMSE'
    }
    
    model = CatBoostRegressor(**params)
    
    model.fit(
        X_opt_train, y_opt_train,
        eval_set=[(X_opt_val, y_opt_val)],
        early_stopping_rounds=100
    )
    
    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print("   -> Optimizing CatBoost...")
study_cat = optuna.create_study(direction='minimize')
study_cat.optimize(objective_cat, n_trials=N_TRIALS)

# --- C. LightGBM Optimization (New Callback API) ---
def objective_lgbm(trial):
    params = {
        'n_estimators': 4000,
        'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.1, log=True),
        'num_leaves': trial.suggest_int('num_leaves', 30, 200),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.6, 0.95),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.6, 0.95),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.1, 10),
        'reg_lambda': trial.suggest_float('reg_lambda', 0.1, 10),
        'device': 'gpu', # Assicurati di avere LGBM compilato per GPU, altrimenti usa 'cpu'
        'n_jobs': 1,
        'verbosity': -1,
        'random_state': 42,
        'metric': 'rmse'
    }
    
    model = LGBMRegressor(**params)
    
    # NEW API: Usiamo callbacks invece di early_stopping_rounds in .fit()
    callbacks = [
        early_stopping(stopping_rounds=100, verbose=False),
        log_evaluation(period=0) # Zittisce l'output
    ]
    
    try:
        model.fit(
            X_opt_train, y_opt_train,
            eval_set=[(X_opt_val, y_opt_val)],
            eval_metric='rmse',
            callbacks=callbacks
        )
    except Exception as e:
        # Fallback CPU se GPU crasha o non presente
        params['device'] = 'cpu'
        model = LGBMRegressor(**params)
        model.fit(
            X_opt_train, y_opt_train,
            eval_set=[(X_opt_val, y_opt_val)],
            eval_metric='rmse',
            callbacks=callbacks
        )

    preds = model.predict(X_opt_val)
    return np.sqrt(mean_squared_error(y_opt_val, preds))

print("   -> Optimizing LightGBM...")
study_lgbm = optuna.create_study(direction='minimize')
study_lgbm.optimize(objective_lgbm, n_trials=N_TRIALS)

# Recupero i migliori parametri
best_xgb_params = study_xgb.best_params
best_xgb_params.update({'n_estimators': 2000, 'tree_method': 'hist', 'device': 'gpu', 'n_jobs': 1, 'random_state': 42})
# Rimuoviamo early_stopping_rounds dai params per il fit finale (opzionale, ma pulito)
if 'early_stopping_rounds' in best_xgb_params: del best_xgb_params['early_stopping_rounds']

best_cat_params = study_cat.best_params
best_cat_params.update({'iterations': 2000, 'task_type': 'GPU', 'devices': '0', 'verbose': 0, 'random_state': 42})

best_lgbm_params = study_lgbm.best_params
best_lgbm_params.update({'n_estimators': 2000, 'device': 'gpu', 'n_jobs': 1, 'verbosity': -1, 'random_state': 42})

print("\n--- Optuna Completato. Parametri migliori trovati. ---")

# ---------------------------------------------------------
# 5. SMART SOFT MODEL SELECTOR (Mixture of Experts)
# ---------------------------------------------------------
print("\n5. Costruzione Smart Soft Selector (Mixture of Experts)...")

class SmartSoftModelSelector(BaseEstimator, RegressorMixin):
    def __init__(self, estimators, selector_model=None):
        self.estimators = estimators # Lista di tuple ('nome', modello)
        # Usiamo XGBClassifier su GPU come selettore per massima velocità
        self.selector_model = selector_model if selector_model else XGBClassifier(
            n_estimators=200, max_depth=6, learning_rate=0.05,
            tree_method='hist', device='gpu', n_jobs=1, random_state=42
        )
        self.model_names = [name for name, _ in estimators]
        
    def fit(self, X, y):
        # A. Addestramento Modelli Base
        print("   -> Training modelli base (Esperti) su tutto il Train Set...")
        self.fitted_estimators_ = []
        for name, model in self.estimators:
            # Nota: Qui non usiamo Early Stopping perché vorremmo usare tutto il train set.
            # Usiamo i parametri ottimizzati da Optuna che sono robusti.
            model.fit(X, y)
            self.fitted_estimators_.append(model)
            
        # B. Generazione OOF Predictions (Chi sbaglia meno dove?)
        print("   -> Generazione dati per il Gating Network...")
        errors = pd.DataFrame()
        
        # Nota: Usiamo n_jobs=1 in cross_val_predict perché i modelli usano GPU internamente
        for name, model in self.estimators:
            # cross_val_predict è essenziale per evitare leakage
            oof_preds = cross_val_predict(model, X, y, cv=5, n_jobs=1)
            errors[name] = np.abs(y - oof_preds) # Errore Assoluto
            
        # C. Creazione Target per il Selettore
        y_best_model_idx = errors.idxmin(axis=1).apply(lambda x: self.model_names.index(x))
        
        # D. Addestramento Selettore (Gating Network)
        print("   -> Training del Selettore (Gating Network)...")
        self.selector_model.fit(X, y_best_model_idx)
        return self

    def predict(self, X):
        # A. Predizioni Base
        base_preds = np.column_stack([model.predict(X) for model in self.fitted_estimators_])
        # B. Pesi Soft
        weights = self.selector_model.predict_proba(X)
        # C. Media Ponderata Dinamica
        final_pred = np.sum(base_preds * weights, axis=1)
        return final_pred

# Istanziamo i modelli base con i Best Params di Optuna
estimators_list = [
    ('xgb', XGBRegressor(**best_xgb_params)),
    ('cat', CatBoostRegressor(**best_cat_params)),
    ('lgbm', LGBMRegressor(**best_lgbm_params)) 
]

# Creiamo e addestriamo il sistema MoE
moe_model = SmartSoftModelSelector(estimators=estimators_list)
moe_model.fit(X_train, y_train)

# ---------------------------------------------------------
# 6. AGGIUNTA DEL CORRETTORE DEI RESIDUI (Residual Learning)
# ---------------------------------------------------------
print("\n6. Training Correttore dei Residui...")

class ResidualCorrectedMoE(BaseEstimator, RegressorMixin):
    def __init__(self, base_moe_model, corrector_model=None):
        self.base_moe_model = base_moe_model
        # Usiamo un CatBoost leggero come correttore
        self.corrector_model = corrector_model if corrector_model else CatBoostRegressor(
            iterations=500,          
            depth=6, 
            learning_rate=0.03,
            l2_leaf_reg=5,           
            task_type='GPU', 
            devices='0',
            verbose=0,
            allow_writing_files=False,
            random_state=42
        )
        
    def fit(self, X, y):
        
        # 2. Generiamo le predizioni OOF per calcolare i residui onesti
        print("   -> Calcolo residui OOF (può richiedere tempo)...")
        oof_preds = cross_val_predict(self.base_moe_model, X, y, cv=5, n_jobs=1)
        
        # 3. Calcolo dell'Errore
        residuals = y - oof_preds
        
        print(f"      Media Residui: {residuals.mean():.4f}")
        print(f"      Deviaz. Std Residui: {residuals.std():.4f}")
        
        # 4. Addestriamo il Correttore
        print("   -> Training del Correttore...")
        self.corrector_model.fit(X, residuals)
        return self

    def predict(self, X):
        base_pred = self.base_moe_model.predict(X)
        correction = self.corrector_model.predict(X)
        return base_pred + (1.0 * correction)

# Costruzione Finale
# Nota: passiamo il moe_model già addestrato, ma la classe ResidualCorrectedMoE
# userà cross_val_predict che internamente farà cloni e fit su fold.
final_system = ResidualCorrectedMoE(base_moe_model=moe_model)
final_system.fit(X_train, y_train)

# Valutazione
y_pred_corrected = final_system.predict(X_test)

final_rmse_corr = np.sqrt(mean_squared_error(y_test, y_pred_corrected))
final_r2_corr = r2_score(y_test, y_pred_corrected)

print(f"\n==========================================")
print(f" RISULTATI FINALI OTTIMIZZATI")
print(f"==========================================")
print(f" RMSE: {final_rmse_corr:.5f}")
print(f" R^2 : {final_r2_corr:.5f}")
print(f"==========================================")

# Plot veloce
corrections_test = final_system.corrector_model.predict(X_test)
base_preds_test = final_system.base_moe_model.predict(X_test)
plt.figure(figsize=(10, 6))
sns.scatterplot(x=base_preds_test, y=corrections_test, alpha=0.3)
plt.axhline(0, color='red', linestyle='--')
plt.title("Bias Correction vs Price (Optimized Models)")
plt.xlabel("Predicted Price")
plt.ylabel("Correction")
plt.show()